# Investigating the temperature distribution along a bar

## Konstantinos Doran SN: 22007700

### Introduction

In this notebook I will be investigating the heat equation in two forms. The first is the steady state solution and the second the time-varying solution. For our steady-state solution I will use an iterative approach to the elliptic equation:
$$
\frac{\partial^2 \theta}{\partial x^2} + \frac{\partial^2 \theta}{\partial y^2} = 0
$$
To solve this, I will implement the successive over-relaxation (SOR) update. Which finds a value $\phi_{i,j}$ at an iteration k+1 from its surrounding points using the following equation:
$$
\phi_{i,j}^{(k+1)} = \frac{\omega}{4} (\phi_{i+1,j}^{(k)}+\phi_{i-1,j}^{(k)}+\phi_{i,j+1}^{(k)}+\phi_{i,j-1}^{(k)}) + (1-\omega)\phi_{i,j}^{(k)}
$$
where $\omega$ is between 0 and 1 and is the relaxation parameter. 
NOTE:
All the plots DO NOT calculate what occurs at the boundaries of each plot.

In [ ]:
# Appropriate imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### 1. Set up the SOR solver

In [ ]:
def update_GS_SOR(phi, M, N, omega):
    """Update MxN grid of phi using SOR and Gauss-Seidel method
    Inputs:
    phi     array of temperature with dimension [N,M]
    M       size of array in one dimension
    N       size of array in second dimension
    omega   relaxation parameter
    Outputs:
    phi     updated array of temperature with dimension [N,M]
    """
    # We exclude i=0, i=M, j=0 and j=N as they are boundaries
    for i in range(1,N-1):
        for j in range(1,M-1):
            phi[i,j] = omega*0.25*(phi[i-1,j] + phi[i+1,j] + phi[i,j-1] + phi[i,j+1]) + (1 - omega)*phi[i,j]
    return phi

### 2. Set up the array and initial conditions

In [ ]:
# Initialise array and boundary conditions
M = 42
N = 30                      
T1 = 300                    #celsius
T2 = 400                    #celsius
theta = T1 * np.ones( (N,M) )
theta[ :,0] = T2
#print(theta)
# Plot initial guess
plt.imshow(theta,origin='lower', extent=[0,21,0,15])
plt.colorbar(label='Temperature (C)')
plt.xlabel('length in m')
plt.ylabel('width in m')
plt.title('Guess Temperature distribution along bar')

### 3. Run the solver and plot the result
First I will look at the effect of changing the array size before plotting the result and how the size and also our value for omega affects the time it takes to calculate our distribution.

In [ ]:
# Initialise array and boundary conditions
M = 21
N = 15
T1 = 300                    #celsius
T2 = 400                    #celsius
theta = T1 * np.ones( (N,M) )
theta[ :,0] = T2
# Set parameter values for while loop
omega = 0.5
delta = 1
tol = 1e-3
iter = 1
maxiter = 1000
while delta > tol and iter < maxiter:
    # Calculate new theta array
    thetain = np.copy(theta)
    theta = update_GS_SOR(theta, M, N, omega)
    # Compare to previous array
    delta = np.max(np.abs(thetain - theta))
    iter += 1
print("Finished after ",iter," iterations")


With array size of 1 element per meter, our solver requires 463 iterations, which is a large amount. When using omega = 1 the number of iterations is cut down to below 200 which saves a lot more time and requires less computational resources.

In [ ]:
# Initialise array and boundary conditions
M = 42
N = 30
T1 = 300                    #celsius
T2 = 400                    #celsius
theta = T1 * np.ones( (N,M) )
theta[ :,0] = T2
# Set parameter values for while loop
omega = 0.5
delta = 1
tol = 1e-3
iter = 1
maxiter = 1000
while delta > tol and iter < maxiter:
    # Calculate new theta array
    thetain = np.copy(theta)
    theta = update_GS_SOR(theta, M, N, omega)
    # Compare to previous array
    delta = np.max(np.abs(thetain - theta))
    iter += 1
print("Finished after ",iter," iterations")

With omega still at 0.5 but the array size becoming 4 times larger, it reaches the maximum iterations before finding a stable distribution. This indicates that we need to increase our value of omega so that it is less relaxed and will take fewer iterations.

In [ ]:
# Initialise array and boundary conditions
M = 42
N = 30
T1 = 300                    #celsius
T2 = 400                    #celsius
theta = T1 * np.ones( (N,M) )
theta[:,0] = T2
# Set parameter values for while loop
omega = 1
delta = 1
tol = 1e-3
iter = 1
maxiter = 1000
while delta > tol and iter < maxiter:
    # Calculate new theta array
    thetain = np.copy(theta)
    theta = update_GS_SOR(theta, M, N, omega)
    # Compare to previous array
    delta = np.max(np.abs(thetain - theta))
    iter += 1
print("Finished after ",iter," iterations")

With omega set to one, the over-relaxation parameter can be discarded. It only increases the number of iterations and now our SOR solver becomes a normal Gauss-Seidel Solver and completes the distribution calculation in the fewest number of iterations.

In [ ]:
# Plot final distribution using SOR method
plt.imshow(theta,origin='lower', extent=[0,21,0,15])
plt.colorbar(label='Temperature (C)')
plt.xlabel('length in m')
plt.ylabel('width in m')
plt.title('Final Temperature distribution along bar')

### 4. Time evolution function
Now looking at the time evolving solution, our method for calculating each point in the array is similar and uses the following equation:
$$
\theta_{i,j,n+1} = \theta_{i,j,n} + \zeta ( \theta_{i+1,j,n} + \theta_{i-1,j,n} + \theta_{i,j+1,n} + \theta_{i,j-1,n} - 4 \theta_{i,j,n})
$$
With n being the current timestep and i and j the x and y position in the array. 

In [ ]:
def update_temperature2D(temper, M, N, zeta):
    """Perform explicit forward FD update for heat equation
    Inputs:
    temper  Array of temperature at present timestep
    n       Temperature timestep to calculate
    zeta    Constant of proportionality
    Outputs:
    temper_next Array of temperature at next timestep
    """
    # How do you initialise temper_next to preserve boundary conditions?
    temper_next = np.zeros((N,M))
    temper_next[:,0] =temper[:,0]
    temper_next[:,-1] = temper[:,-1]
    temper_next[0,:] = temper[0,:]
    temper_next[-1,:] = temper[-1,:]
    for i in range(1, N-1): # Don't update end-points
        for j in range(1, M-1):
            temper_next[i,j] = temper[i,j] + zeta*(temper[i+1,j] + temper[i-1,j] + temper[i,j+1] + temper[i,j-1] - 4*temper[i,j])
    return temper_next

### 5. Set up initial conditions

In [ ]:
# Initialise array and boundary conditions
M = 42
N = 30
T1 = 300                    #celsius
T2 = 400                    #celsius
theta0 = T1*np.ones((N,M))
theta0[:,0] = T2 
# Plot initial guess
plt.imshow(theta0,origin='lower', extent=[0,21,0,15])
plt.colorbar(label='Temperature (C)')
plt.xlabel('length in m')
plt.ylabel('width in m')
plt.title('Initial Temperature distribution along bar')

### 6. Propagate the solution and store selected points

In [ ]:
# Initialise zeta and number of time steps
Nt = 400
zeta = 0.1
for i in range(1,Nt):
    # Use new distribution to calculate next one
    theta_next = update_temperature2D(theta0, M, N, zeta)
    theta0 = theta_next

    
#Plot final distribution    
plt.imshow(theta0,origin='lower',extent=[0,21,0,15])
plt.colorbar(label='Temperature (C)')
plt.xlabel('length in m')
plt.ylabel('width in m')
plt.title('Final Temperature distribution along bar')

### Plot the final and steady-state solutions
Below, we also plot the difference between the two solutions to see in detail the difference between the elliptic and the parabolic equations.

In [ ]:
# Plot the difference between the two solutions
difftheta = theta0 - theta
plt.imshow(difftheta,origin='lower', extent=[0,21,0,15])
plt.colorbar(label='Temperature (C)')
plt.xlabel('length in m')
plt.ylabel('width in m')
plt.title('Final Temperature distribution along bar')

## Conclusion
When we increase the size of the array, it is evident that the two solutions differ towards the central sections of the bar, however, if we are to approximate the bar to be thinner, or have a smaller array size the difference between the two solutions would be smaller. Interestingly, the temperature around the boundaries is almost equal or equal and only differs where the temperature difference changes rapidly in the initial distribution. Also the time varying solution took less time to run compared to the iterative approach of the SOR solver.